# *<span style="color:blue">Data Insertion</span>*

In [1]:
import pandas as pd
import pyodbc

In [2]:
df = pd.read_csv("cleaned_data.csv")

In [3]:
server_name = 'LAPTOP-QRK184LV'
database_name = 'jobs'
driver_name = 'ODBC Driver 17 for SQL Server'
conn_string = f'Driver={{ODBC Driver 17 for SQL Server}};Server={server_name};Database={database_name};Trusted_Connection=yes;'


### *<span style="color:green">Tables creation</span>*

In [4]:
server = server_name
database = database_name
driver = driver_name
conn_str = conn_string


conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

# Create tables in the correct order to avoid dependency issues
cursor.execute('''
CREATE TABLE Facility (
    FacilityID INT PRIMARY KEY IDENTITY(1,1),
    FacilityName VARCHAR(255)
)
''')

cursor.execute('''
CREATE TABLE Domain (
    DomainID INT PRIMARY KEY IDENTITY(1,1),
    DomainName VARCHAR(255)
)
''')

cursor.execute('''
CREATE TABLE Requirement (
    RequirementID INT PRIMARY KEY IDENTITY(1,1),
    RequirementName VARCHAR(255)
)
''')

cursor.execute('''
CREATE TABLE Company (
    CompanyID INT PRIMARY KEY IDENTITY(1,1),
    CompanyName VARCHAR(255)
)
''')

cursor.execute('''
CREATE TABLE Country (
    CountryID INT PRIMARY KEY IDENTITY(1,1),
    CountryName VARCHAR(255)
)
''')

cursor.execute('''
CREATE TABLE Experience_Level (
    LevelID INT PRIMARY KEY IDENTITY(1,1),
    LevelName VARCHAR(255)
)
''')

cursor.execute('''
CREATE TABLE Job (
    JobID INT PRIMARY KEY IDENTITY(1,1),
    JobTitle VARCHAR(255),
    JobType VARCHAR(255),
    LevelID INT,
    CountryID INT,
    CompanyID INT,
    Salary FLOAT,
    JobFormat VARCHAR(255),
    CONSTRAINT FK_Job_CompanyID FOREIGN KEY (CompanyID) REFERENCES Company (CompanyID),
    CONSTRAINT FK_Job_LevelID FOREIGN KEY (LevelID) REFERENCES Experience_Level (LevelID),
    CONSTRAINT FK_Job_CountryID FOREIGN KEY (CountryID) REFERENCES Country (CountryID)
)
''')

cursor.execute('''
CREATE TABLE Job_Domain (
    JobID INT,
    DomainID INT,
    CONSTRAINT FK_Job_Domain_JobID FOREIGN KEY (JobID) REFERENCES Job (JobID),
    CONSTRAINT FK_Job_Domain_DomainID FOREIGN KEY (DomainID) REFERENCES Domain (DomainID)
)
''')

cursor.execute('''
CREATE TABLE Job_Requirement (
    JobID INT,
    RequirementID INT,
    CONSTRAINT FK_Job_Requirement_JobID FOREIGN KEY (JobID) REFERENCES Job (JobID),
    CONSTRAINT FK_Job_Requirement_RequirementID FOREIGN KEY (RequirementID) REFERENCES Requirement (RequirementID)
)
''')

cursor.execute('''
CREATE TABLE Job_Facility (
    JobID INT,
    FacilityID INT,
    CONSTRAINT FK_Job_Facility_JobID FOREIGN KEY (JobID) REFERENCES Job (JobID),
    CONSTRAINT FK_Job_Facility_FacilityID FOREIGN KEY (FacilityID) REFERENCES Facility (FacilityID)
)
''')

# Commit the changes and close the connection
conn.commit()
#conn.close()



### *<span style="color:green">Insert data to Requirement Table</span>*

In [ ]:
# Concatenate all the keywords from the entire 'Company_requirement' column into a single string
distinct_requirements_str = ','.join(df['Company_requirement'])

# Extract keywords within two double quotes and remove any empty values
distinct_requirements_list = [req.strip() for req in distinct_requirements_str.replace('"', '').split(',') if req.strip()]

# Convert the list to a set to remove duplicates, then back to a list to maintain order
distinct_requirements_list = list(set(distinct_requirements_list))

# Create a DataFrame with unique 'RequirementName'
requirements_df = pd.DataFrame({'RequirementName': distinct_requirements_list})

# Generate a unique ID ('RequirementID') for each distinct requirement
requirements_df['RequirementID'] = range(1, len(requirements_df) + 1)

# Display the new dataset with 'RequirementName' and 'RequirementID'
display(requirements_df)


In [6]:
# Insert into the Requirement table
for index, row in requirements_df.iterrows():
    cursor.execute("INSERT INTO Requirement (RequirementName) VALUES (?)", row['RequirementName'])
conn.commit()

### *<span style="color:green">Insert data to Facility Table</span>*

In [ ]:
# Concatenate all the keywords from the entire 'Facilities' column into a single string
distinct_facilities_str = ','.join(df['Facilities'])

# Extract keywords within two double quotes and remove any empty values
distinct_facilities_list = [facility.strip() for facility in distinct_facilities_str.replace('"', '').split(',') if facility.strip()]

# Convert the list to a set to remove duplicates, then back to a list to maintain order
distinct_facilities_list = list(set(distinct_facilities_list))

# Create a DataFrame with unique 'FacilityName'
facilities_df = pd.DataFrame({'FacilityName': distinct_facilities_list})

# Generate a unique ID ('FacilityID') for each distinct facility
facilities_df['FacilityID'] = range(1, len(facilities_df) + 1)

# Display the new dataset with 'FacilityName' and 'FacilityID'
display(facilities_df)


In [8]:
# Insert into the Facility table
for index, row in facilities_df.iterrows():
    cursor.execute("INSERT INTO Facility (FacilityName) VALUES (?)", row['FacilityName'])
conn.commit()

### *<span style="color:green">Insert data to Company Table</span>*

In [ ]:

# Create a mapping of unique company names to unique IDs
company_mapping = {}
next_company_id = 1

for company in df['Company'].unique():
    if company not in company_mapping:
        company_mapping[company] = next_company_id
        next_company_id += 1

# Add a new column for CompanyID
df['CompanyID'] = df['Company'].map(company_mapping)

# Now your dataframe contains the CompanyID for each company name
display(df)


In [ ]:
distinct_company_df = df[['Company']].copy().drop_duplicates()
distinct_company_df['CompanyID'] = range(1, len(distinct_company_df) + 1)

# Display the new datasets with unique IDs for each column
print("Distinct Company DataFrame:")
display(distinct_company_df)

In [11]:
# Insert into the Company table
for index, row in distinct_company_df.iterrows():
    cursor.execute("INSERT INTO Company (CompanyName) VALUES (?)", row['Company'])
conn.commit()

### *<span style="color:green">Insert data to Experience_Level Table</span>*

In [ ]:
# Experience level column
distinct_experience_level_df = df[['Experience level']].copy().drop_duplicates()
distinct_experience_level_df['ExperienceLevelID'] = range(1, len(distinct_experience_level_df) + 1)

print("\nDistinct Experience Level DataFrame:")
display(distinct_experience_level_df)


In [13]:
# Insert into the Experience_Level table
for index, row in distinct_experience_level_df.iterrows():
    cursor.execute("INSERT INTO Experience_Level (LevelName) VALUES (?)", row['Experience level'])
conn.commit()

### *<span style="color:green">Insert data to Country Table</span>*

In [ ]:
# country column
distinct_country_df = df[['country']].copy().drop_duplicates()
distinct_country_df['CountryID'] = range(1, len(distinct_country_df) + 1)

print("\nDistinct Country DataFrame:")
display(distinct_country_df)

In [15]:
# Insert into the Country table
for index, row in distinct_country_df.iterrows():
    cursor.execute("INSERT INTO Country (CountryName) VALUES (?)", row['country'])
conn.commit()

### *<span style="color:green">Insert data to Domain Table</span>*

In [ ]:
# Domain column
distinct_domain_df = df[['Domain']].copy().drop_duplicates()
distinct_domain_df['DomainID'] = range(1, len(distinct_domain_df) + 1)

print("\nDistinct Domain DataFrame:")
display(distinct_domain_df)


In [17]:
# Insert into the Domain table
for index, row in distinct_domain_df.iterrows():
    cursor.execute("INSERT INTO Domain (DomainName) VALUES (?)", row['Domain'])
conn.commit()

### *<span style="color:green">Insert data to Job Table</span>*

In [ ]:
# Job Title column
distinct_job_title_df = df[['Job Title']].copy().drop_duplicates()
distinct_job_title_df['JobTitleID'] = range(1, len(distinct_job_title_df) + 1)

print("\nDistinct Job Title DataFrame:")
display(distinct_job_title_df)

In [19]:

# Function to get the ID from the Company table or insert a new company and return the newly generated ID
def get_company_id(company_name):
    cursor.execute("SELECT CompanyID FROM Company WHERE CompanyName=?", company_name)
    company_id_result = cursor.fetchone()
    if company_id_result is not None:
        return company_id_result[0]
    else:
        cursor.execute("SELECT SCOPE_IDENTITY()")
        return cursor.fetchone()[0]

# Function to get the ID from the Country table or insert a new country and return the newly generated ID
def get_country_id(country_name):
    cursor.execute("SELECT CountryID FROM Country WHERE CountryName=?", country_name)
    country_id_result = cursor.fetchone()
    if country_id_result is not None:
        return country_id_result[0]
    else:
        cursor.execute("SELECT SCOPE_IDENTITY()")
        return cursor.fetchone()[0]

# Function to get the ID from the Experience_Level table or insert a new level and return the newly generated ID
def get_level_id(level_name):
    cursor.execute("SELECT LevelID FROM Experience_Level WHERE LevelName=?", level_name)
    level_id_result = cursor.fetchone()
    if level_id_result is not None:
        return level_id_result[0]
    else:
        cursor.execute("SELECT SCOPE_IDENTITY()")
        return cursor.fetchone()[0]

# Insert data into the Job table
for index, row in df.iterrows():
    company_id = get_company_id(row['Company'])
    country_id = get_country_id(row['country'])
    level_id = get_level_id(row['Experience level'])
    
    cursor.execute("INSERT INTO Job (JobTitle, JobType, LevelID, CountryID, CompanyID, Salary, JobFormat) VALUES (?, ?, ?, ?, ?, ?, ?)",
                   row['Job Title'], row['Job Type'], level_id, country_id, company_id, row['Salary (USD)'], row['Job Format'])

# Commit the changes and close the connection
conn.commit()
# conn.close()


### *<span style="color:green">Insert data to Job_Facility , Job_Domain and Job_Requirement Tables</span>*

In [ ]:
# Fetch job_ids from the Job table
cursor.execute("SELECT JobID FROM Job")
job_ids_result = cursor.fetchall()
job_ids = [row[0] for row in job_ids_result]

# Loop through each row in the DataFrame
for index, row in df.iterrows():
    job_id = job_ids[index]

    # Insert into the Job_Facility table
    for facility in row['Facilities'].split(','):
        cursor.execute("SELECT FacilityID FROM Facility WHERE FacilityName=?", facility.strip())
        facility_id_result = cursor.fetchone()
        if facility_id_result is not None:
            facility_id = facility_id_result[0]
        else:
            # If the FacilityName does not exist in the Facility table, you may choose to insert it or handle it as needed
            # For example, you can insert a new facility entry and get the newly generated FacilityID
            cursor.execute("INSERT INTO Facility (FacilityName) VALUES (?)", facility.strip())
            cursor.execute("SELECT SCOPE_IDENTITY()")
            facility_id = cursor.fetchone()[0]

        cursor.execute("INSERT INTO Job_Facility (JobID, FacilityID) VALUES (?, ?)", job_id, facility_id)

    # Insert into the Job_Domain table
    for domain in row['Domain'].split(','):
        cursor.execute("SELECT DomainID FROM Domain WHERE DomainName=?", domain.strip())
        domain_id_result = cursor.fetchone()
        if domain_id_result is not None:
            domain_id = domain_id_result[0]
            cursor.execute("INSERT INTO Job_Domain (JobID, DomainID) VALUES (?, ?)", job_id, domain_id)
        else:
            print(f"Domain not found in the Domain table: {domain.strip()}")

    # Insert into the Job_Requirement table
    for requirement in row['Company_requirement'].split(','):
        cursor.execute("SELECT RequirementID FROM Requirement WHERE RequirementName=?", requirement.strip())
        requirement_id_result = cursor.fetchone()
        if requirement_id_result is not None:
            requirement_id = requirement_id_result[0]
        else:
            # For example, you can insert a new requirement entry and get the newly generated RequirementID
            cursor.execute("INSERT INTO Requirement (RequirementName) VALUES (?)", requirement.strip())
            cursor.execute("SELECT SCOPE_IDENTITY()")
            requirement_id = cursor.fetchone()[0]

        cursor.execute("INSERT INTO Job_Requirement (JobID, RequirementID) VALUES (?, ?)", job_id, requirement_id)

# Commit the changes and close the connection
conn.commit()
# conn.close()
